In [77]:
import pandas as pd
import matplotlib.pyplot as plt
from urllib.parse import quote
import matplotlib as mpl
import numpy as np

In [93]:
raw_data = pd.read_csv('C://Workspace/02.DataAnalysis/05.Crawling/data/서울시 주요 공원현황.csv', encoding='EUC-KR')

In [79]:
import re

In [89]:
a = raw_data['면적'][6]
type(a),a

(str,
 '휴양 및 편익시설 : 평의자 7개 화장실 23.18㎡ 조경시설 - 은하폭포(높이 12m × 폭10m 발원지로부터 17.3m 폭 1.5 ~ 10m 수조 255㎡ 담수량 140t) - 산성폭포(높이 5m×폭 12m 수조 90㎡ 담수량 75t)- 명속의 벽 1개소(높이 5.4m) 교양시설 : 야외공연무대 102㎡ 관리시설 : 관리실 8.55㎡ 전기실 51.03㎡ 기계실 83.36㎡ 안내판 1개 공원 등 18본 휴양시설 : 벤취 12개 ')

In [81]:
a =raw_data['면적'][15].replace('㎡','')
a

'61544 녹지대 : 40792 광 장 : 6600 기 타 : 14152'

In [26]:
raw_data.index

RangeIndex(start=0, stop=132, step=1)

In [ ]:
import re
area = df.면적.astype(str).apply(lambda x: x.strip('총 ').split()[0])
area[(area == 'nan') | (area == '휴양')] = '0㎡'
area = area.apply(lambda x: float(re.sub('[㎡m²㎥]','',x)))

In [91]:
area = raw_data.면적.astype(str).apply(lambda x: x.strip('총 ').split()[0])
area[(area == 'nan') | (area == '휴양')] = '0㎡'
area = area.apply(lambda x: float(re.sub('[㎡m²㎥]','',x)))



In [92]:
raw_data['면적2'] = area

In [ ]:
df = pd.read_csv('C://Workspace/02.DataAnalysis/05.Crawling/data/서울시 주요 공원현황.csv', encoding='EUC-KR')
df.head()

In [95]:
area = df.면적.astype(str).apply(lambda x: x.strip('총 ').split()[0])
area[-5:]

127    109635㎡
128    140022㎡
129    211392㎡
130    157535㎡
131        nan
Name: 면적, dtype: object

In [96]:
area[(area == 'nan') | (area == '휴양')] = '0㎡'

In [97]:
area = area.apply(lambda x: float(re.sub('[㎡m²㎥]','',x)))

In [100]:
area_criteria = [-1,100000,1000000,12000000]
labels = ['소형','중형','대형']
size_info = [3,7,15]
scale = pd.cut(area, area_criteria,labels=labels)
size = pd.cut(area, area_criteria,labels=size_info)

### 데이터프레임 만들고 저장

In [102]:
df = df[['공원명','지역','X좌표(WGS84)','Y좌표(WGS84)']]
df.columns = ['공원명','지역','경도','위도']
df['면적'] = area
df['분류'] = scale
df['크기'] = size
df.head()

C:\Users\igksj\anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,공원명,지역,경도,위도,면적,분류,크기
0,남산도시자연공원,중구,126.990377,37.550140,2896887.0,대형,15
1,길동생태공원,강동구,127.154779,37.540394,80683.0,소형,3
2,서울대공원,과천시,127.019846,37.426449,9132690.0,대형,15
3,서울숲,성동구,127.041798,37.543072,480994.0,중형,7
4,월드컵공원,마포구,126.878907,37.571805,2284085.0,대형,15


In [103]:
df.isnull().sum()

공원명    0
지역     1
경도     1
위도     1
면적     0
분류     0
크기     0
dtype: int64

In [104]:
df[df.경도.isna()]

,공원명,지역,경도,위도,면적,분류,크기
131,서울로7017,중구,NaN,NaN,0.0,소형,3


In [106]:
df = df.dropna()

In [107]:
df.to_csv('서울공원 정리.csv', index=False)

In [108]:
import folium

In [ ]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11)
for i in df.index:
    folium.CircleMarker([df.위도[i],df.경도[i]],
                radius=int(df.크기[i]),
                tooltip=f'{df.공원명[i]}({df.면적[i]:,d}m²)',
                color='crimson', fill_color='crimson').add_to(map
title_html = '<h3 align="center" style="font-size:20px"><b>서울시내 공원 위치와 크기</b></h3>'   
map.get_root().html.add_child(folium.Element(title_html))
map

In [ ]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')
for i in df.index:
    folium.CircleMarker([df.위도[i], df.경도[i]], 
                        radius=int(df.크기[i]),
                        tooltip=f"{df.공원명[i]}({df.면적[i]:,d}㎡)",
                        color='green', fill_color='green').add_to(map)
title_html = '<h3 align="center" style="font-size:20px"><b>서울시내 공원 위치와 크기</b></h3>'   
map.get_root().html.add_child(folium.Element(title_html))
map

### 자치구 공원 분석
- 자치구별로 공원면적과 공원수를 집계하시오.
- 서울시 통계를 이용해 구별 공원면적 비율과 인당 공원면적을  구하여 막대그래프로 표시하시오.

In [113]:
df = pd.read_csv('서울공원 정리.csv')

In [114]:
df.head()

,공원명,지역,경도,위도,면적,분류,크기
0,남산도시자연공원,중구,126.990377,37.550140,2896887.0,대형,15
1,길동생태공원,강동구,127.154779,37.540394,80683.0,소형,3
2,서울대공원,과천시,127.019846,37.426449,9132690.0,대형,15
3,서울숲,성동구,127.041798,37.543072,480994.0,중형,7
4,월드컵공원,마포구,126.878907,37.571805,2284085.0,대형,15


In [119]:
gu_df = df.groupby(df.지역)[['면적']].sum()

In [122]:
gu_df['공원수'] = df['지역'].value_counts()

In [194]:
gu_df

,면적,공원수
지역,,
강남구,3626398.50,7
강동구,1526389.30,7
강북구,3714902.40,4
강서구,2194154.10,9
과천시,9132690.00,1
관악구,10431453.08,2
광진구,584002.00,2
구로구,1617657.00,4
금천구,57568.00,4


In [130]:
people_df = pd.read_table('report.txt',sep='\t')

In [195]:
people_df

,기간,지역,인구,면적,인구밀도(명/㎢)
0,2020,합계,"9,911,088",605.23,"16,376"
1,2020,종로구,"158,996",23.91,"6,649"
2,2020,중구,"134,635",9.96,"13,517"
3,2020,용산구,"244,645",21.87,"11,188"
4,2020,성동구,"300,505",16.86,"17,822"
5,2020,광진구,"360,109",17.06,"21,105"
6,2020,동대문구,"357,014",14.22,"25,114"
7,2020,중랑구,"399,562",18.50,"21,603"
8,2020,성북구,"447,056",24.58,"18,190"
9,2020,강북구,"311,569",23.60,"13,202"


In [135]:
type(people_df)

pandas.core.frame.DataFrame

In [200]:
gu_p_df = people_df[['지역','인구','면적']]
gu_p_df

,지역,인구,면적
0,합계,"9,911,088",605.23
1,종로구,"158,996",23.91
2,중구,"134,635",9.96
3,용산구,"244,645",21.87
4,성동구,"300,505",16.86
5,광진구,"360,109",17.06
6,동대문구,"357,014",14.22
7,중랑구,"399,562",18.50
8,성북구,"447,056",24.58
9,강북구,"311,569",23.60


In [201]:
gu_p_df.set_index('지역', inplace=True)

In [ ]:
gu_p_df.columns = ['지역','인구','지역면적']

In [202]:
gu_p_df

,인구,면적
지역,,
합계,"9,911,088",605.23
종로구,"158,996",23.91
중구,"134,635",9.96
용산구,"244,645",21.87
성동구,"300,505",16.86
광진구,"360,109",17.06
동대문구,"357,014",14.22
중랑구,"399,562",18.50
성북구,"447,056",24.58


In [203]:
result = pd.concat([gu_df,gu_p_df],axis=1,join='inner')
result

,면적,공원수,인구,면적
지역,,,,
강남구,3626398.50,7,"544,055",39.50
강동구,1526389.30,7,"463,998",24.59
강북구,3714902.40,4,"311,569",23.60
강서구,2194154.10,9,"585,901",41.44
관악구,10431453.08,2,"509,803",29.57
광진구,584002.00,2,"360,109",17.06
구로구,1617657.00,4,"432,488",20.12
금천구,57568.00,4,"247,835",13.02
노원구,12024307.30,2,"527,032",35.44


In [205]:
result.columns = ['공원면적','공원수','인구','지역면적']

In [173]:
result['공원면적'].sum()

83848323.27000001

In [206]:
result['공원면적 비율'] = (result.공원면적/ result['공원면적'].sum()) * 100

In [ ]:
result['인구'] = result.인구.aply(lambda x: replace(',',''))

In [ ]:
result=result.astype('float')

In [ ]:
result['공원면적\/인구'] = result.공원면적/result.인구

In [208]:
result.head()

,공원면적,공원수,인구,지역면적,공원면적 비율
지역,,,,,
강남구,3626398.50,7,"544,055",39.50,4.324951
강동구,1526389.30,7,"463,998",24.59,1.820417
강북구,3714902.40,4,"311,569",23.60,4.430503
강서구,2194154.10,9,"585,901",41.44,2.616813
관악구,10431453.08,2,"509,803",29.57,12.440861
